# Load required modules

In [ ]:
# MongoDB
from pymongo import MongoClient
from pymongo.database import Database
import pandas as pd

# PostgreSQL
import psycopg2  # handles database connection to postgres db
import pandas.io.sql as sqlio  # we'll use this for improved query presentation

# Connect to the MongoDB Atlas Cluster <img src="mongodb.png" width=120 align="right">

In [ ]:
# to be deleted before push!
connection_string = "put your string here"

client = MongoClient(connection_string)
mw243 = Database(client, 'mw243')  # connect to the mw243 database

# Connect to the relational database <img src="postgres.png" width=40 align="right">

In [ ]:
# to be deleted before push!
conn = psycopg2.connect(host="XXX",
                        database="XX", 
                        user="XX", 
                        password="XX",  # password provided in lecture
                        options=f'-c search_path=XX')

conn

# Insert a JSON document to a new database mw243 (collection users)

In [ ]:
# note: this is redundant - we already added "sue" via NoSQLBooster
doc = {
    'name' : 'sue',
    'age' : 26,
    'status' : 'A',
}

result = mw243.users.insert_one(doc) # insert the document into the collection users which is automaticall created
result.inserted_id  # get the id of the new document to see if everything is fine 

<img src="exercise.png">

# Exercise 3.1a <img src="mongodb.png" width=120 align="right">
Add three further users with the same schema, i.e. name, age, and status.

In [ ]:
doc = {
    'name' : 'pete',
    'age' : 34,
    'status' : 'A',
}


result = mw243.users.insert_one(doc) 
result.inserted_id

In [ ]:
docs = [{
    'name' : 'dave',
    'age' : 22,
    'status' : 'B',
},
    {
    'name' : 'elen',
    'age' : 29,
    'status' : 'C',
}
]

result = mw243.users.insert_many(docs) 
result.inserted_ids

## Check insertion of documents

In [ ]:
cursor = mw243.users.find()

df = pd.DataFrame(list(cursor))  # convert to a dataframe for better visualization
df

# Exercise 3.1b <img src="postgres.png" width=40 align="right">
Create a table <code>users</code> in your schema <code>mw240_XXXXXX</code> in the postgres database!<br>
Please note: we set the schema as default when connecting (see above), so you don't have to explicitly set the schema. Just make sure that the parameter <code>search_path</code> is set to <code>mw240_XXXXXX</code> (with XXXXXX = your matriculation number).

## Table creation<img src="postgres.png" width=40 align="right">

In [ ]:
sql = """
CREATE TABLE users (
    user_id serial primary key,
    name varchar(50),
    age integer,
    status varchar(10)
)
"""

cur = conn.cursor()
try:
    cur.execute(sql)
except Exception as e:
    print("Error creating the table: {}".format(e))

conn.commit()

# Insert<img src="postgres.png" width=40 align="right">

In [ ]:
sql = """
INSERT INTO users (name, age, status) values ('sue', 26, 'A')
"""

cur = conn.cursor()
try:
    cur.execute(sql)
except Exception as e:
    print("Error inserting into table: {}".format(e))

conn.commit()

## Check insert<img src="postgres.png" width=40 align="right">

In [ ]:
# Select all elements from table users
sql = """
SELECT * FROM users
"""

dat = sqlio.read_sql_query(sql, conn)
dat

# Exercise 3.1c <img src="mongodb_to_postgres.png" width=200 align="right">
We now want to copy all documents from our MongoDB collection <code>mw243.users</code> to the PostgreSQL table <code>users</code>. Please note: we assume that all documents have the keys "name", "age", and "status" (and only these three key-value pairs!).

## Loop through the pymongo cursor object and insert into the relational database<img src="mongodb_to_postgres.png" width=120 align="right">

In [ ]:
cursor = mw243.users.find()

for doc in cursor:
    sql = "INSERT INTO users (name, age, status) VALUES ('{}', {}, '{}')".format(doc['name'], doc['age'], doc['status'])
    cur = conn.cursor()
    try:
        cur.execute(sql)
    except Exception as e:
        print("Error inserting into table: {}".format(e))

    conn.commit()

## Check insertion of all documents<img src="postgres.png" width=40 align="right">

In [ ]:
# Select all elements from table users
sql = """
SELECT * FROM users
"""

dat = sqlio.read_sql_query(sql, conn)
dat

# Exercise 3.1d <img src="postgres_to_mongodb.png" width=200 align="right">
We now want to copy a single row our PostgreSQL table <code>users</code> to the MongoDB collection mw243.users<br>
Preparation: insert the following users:
- name <code>steve</code>, age <code>33</code>, status <code>D</code> 
- name <code>helen</code>, age <code>21</code>, status <code>D</code> 

## Insert users table<img src="postgres.png" width=40 align="right">

In [ ]:
sql = """INSERT INTO users (name, age, status) VALUES ('steve', 33, 'D'), ('helen', 21, 'D')"""

cur = conn.cursor()
try:
    cur.execute(sql)
except Exception as e:
    print("Error inserting into table: {}".format(e))

conn.commit()

## Check insertion of status 'D'-users in table users<img src="postgres.png" width=40 align="right">

In [ ]:
# Select all elements from table users
sql = """
SELECT * FROM users
"""

dat = sqlio.read_sql_query(sql, conn)
dat

## Programmatically transfer status 'D' rows from PostgreSQL to MongoDB<img src="postgres_to_mongodb.png" width=120 align="right">

In [ ]:
sql = """
SELECT * FROM users WHERE status='D'
"""

dat = sqlio.read_sql_query(sql, conn)

for row in dat.values:
    insert_dict = {'name':row[1], 'age':row[2], 'status':row[3]}
    result = mw243.users.insert_one(insert_dict)
    print(result.inserted_id)

## Check insertion of documents in MongoDB<img src="mongodb.png" width=120 align="right">

In [ ]:
cursor = mw243.users.find()

df = pd.DataFrame(list(cursor))  # convert to a dataframe for better visualization
df

# Cleaning up / Resetting the Status
## Use this to delete / reset the MongoDB collection

In [ ]:
result = mw243.users.delete_many({})
result.deleted_count

## Use this to empty the PostgreSQL table

In [ ]:
sql = """
DELETE FROM users
"""

cur = conn.cursor()
try:
    cur.execute(sql)
except Exception as e:
    print("Error deleting from table: {}".format(e))

conn.commit()

## Use this to drop the PostgreSQL table

In [ ]:
sql = """
DROP TABLE users
"""

cur = conn.cursor()
try:
    cur.execute(sql)
except Exception as e:
    print("Error dropping from table: {}".format(e))

conn.commit()

# Exercise 3.2

## Filtering documents with MongoDB<img src="mongodb.png" width=120 align="right">

In [ ]:
filt = {"$and":
        [
            {"$or":
             [
                 {"status":"A"},
                 {"status":"D"}
             ]
            },
            {"age":{"$gt":22}}
        ]
       }
cursor = mw243.users.find(filt)

df = pd.DataFrame(list(cursor))  # convert to a dataframe for better visualization
df

## Filtering rows in SQL<img src="postgres.png" width=40 align="right">

In [ ]:
sql = """
SELECT * FROM users WHERE (status='A' or status='D') and age > 22
"""

dat = sqlio.read_sql_query(sql, conn)
dat

# Exercise 3.3

## Updating documents with MongoDB<img src="mongodb.png" width=120 align="right">

In [ ]:
filt = {"name":"sue"}
update = {"$set" : {"age":30}}
result = mw243.users.update_one(filt, update)

result.modified_count

# Exercise 3.4

## Deleting documents with MongoDB<img src="mongodb.png" width=120 align="right">

In [ ]:
filt = {"age": {"$gt":30}}
result = mw243.users.delete_many(filt)

result.deleted_count